# Searching Algorithms

*Finding elements efficiently in data structures*


## 🎯 Learning Objectives

- Implement linear search and understand when it's the appropriate choice
- Master binary search and its variants: finding boundaries (first/last occurrence), searching rotated arrays
- Apply the 'binary search on answer' pattern to optimization problems
- Search efficiently in 2D sorted arrays, trees, and graphs
- Use Python's bisect module for production-quality binary search operations

---
## 1. Introduction to Searching


Searching is a fundamental operation—finding a target element or determining its absence. The choice of algorithm depends on whether data is sorted, the data structure used, and whether we need exact matches or ranges. Efficient searching is crucial for databases, file systems, and countless applications.

> 📖 **Definition:** A **search algorithm** finds the position of a target value within a data structure. Key metrics include time complexity, space complexity, and whether the algorithm requires sorted data. Search can return an index, a boolean, or the element itself.

### Search Algorithm Comparison

| Algorithm | Time | Space | Requires Sorted | Data Structure |
| --- | --- | --- | --- | --- |
| Linear Search | O(n) | O(1) | No | Any sequence |
| Binary Search | O(log n) | O(1) | Yes | Array |
| Hash Table | O(1) avg | O(n) | No | Hash table |
| BST Search | O(log n) avg | O(1) | Inherent | BST |
| BFS/DFS | O(V + E) | O(V) | No | Graph |

**Listing 11.1 — Search Performance Comparison**

In [ ]:
import time
import random

def linear_search(arr, target):
    for i, val in enumerate(arr):
        if val == target:
            return i
    return -1

def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1

# Compare performance
n = 100000
arr = list(range(n))
target = n - 1  # Worst case for linear

# Linear search
start = time.perf_counter()
linear_search(arr, target)
linear_time = time.perf_counter() - start

# Binary search
start = time.perf_counter()
binary_search(arr, target)
binary_time = time.perf_counter() - start

print(f"Searching for {target} in {n} elements:")
print(f"  Linear search: {linear_time*1000:.3f} ms")
print(f"  Binary search: {binary_time*1000:.6f} ms")
speedup = linear_time/binary_time if binary_time > 0 else float('inf')
print(f"  Speedup: {speedup:.0f}x")

***Figure 11.1:** Binary search is exponentially faster than linear search for large sorted arrays.*

---
## 2. Linear Search


> 📖 **Definition:** **Linear Search** (or sequential search) examines each element in order until finding the target or reaching the end. Simple but inefficient for large datasets—O(n) time. Works on any collection, sorted or not.

**Listing 11.2 — Linear Search Variations**

In [ ]:
def linear_search(arr, target):
    """Basic linear search - O(n) time, O(1) space."""
    for i in range(len(arr)):
        if arr[i] == target:
            return i
    return -1

def linear_search_all(arr, target):
    """Find all occurrences of target."""
    indices = []
    for i, val in enumerate(arr):
        if val == target:
            indices.append(i)
    return indices

def linear_search_condition(arr, condition):
    """Find first element satisfying condition."""
    for i, val in enumerate(arr):
        if condition(val):
            return i, val
    return -1, None

# Test
arr = [4, 2, 7, 1, 9, 2, 5, 2]
print(f"Array: {arr}")
print(f"Index of 7: {linear_search(arr, 7)}")
print(f"All indices of 2: {linear_search_all(arr, 2)}")

idx, val = linear_search_condition(arr, lambda x: x > 5)
print(f"First element > 5: {val} at index {idx}")

***Figure 11.2:** Linear search is versatile—can find first, all, or conditional matches.*

**Listing 11.3 — Python's Built-in Search**

In [ ]:
# Python's built-in search methods (all O(n) for lists)

arr = [4, 2, 7, 1, 9, 2, 5]

# in operator - checks existence
print(f"7 in arr: {7 in arr}")
print(f"10 in arr: {10 in arr}")

# index() - returns first index (raises ValueError if not found)
print(f"arr.index(2): {arr.index(2)}")

# count() - counts occurrences
print(f"arr.count(2): {arr.count(2)}")

# Using try/except for safe index lookup
def safe_index(arr, target):
    try:
        return arr.index(target)
    except ValueError:
        return -1

print(f"safe_index(arr, 10): {safe_index(arr, 10)}")

# For sets and dicts, 'in' is O(1)!
s = set(arr)
print(f"\n7 in set: {7 in s}  (O(1))")

***Figure 11.3:** Python's 'in' operator is O(n) for lists but O(1) for sets and dicts.*

---
## 3. Binary Search


> 📖 **Definition:** **Binary Search** repeatedly divides a sorted array in half, comparing the target to the middle element. Each comparison eliminates half the remaining elements, achieving O(log n) time. Requires random access (arrays, not linked lists).

**Listing 11.4 — Binary Search Iterative**

In [ ]:
def binary_search(arr, target):
    """
    Binary Search - O(log n) time, O(1) space.
    Returns index of target, or -1 if not found.
    """
    left, right = 0, len(arr) - 1
    
    while left <= right:
        mid = left + (right - left) // 2  # Avoid overflow
        
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1

# Trace execution
arr = [1, 3, 5, 7, 9, 11, 13, 15]
target = 7
print(f"Array: {arr}")
print(f"Searching for {target}:\n")

left, right = 0, len(arr) - 1
step = 1
while left <= right:
    mid = (left + right) // 2
    print(f"Step {step}: left={left}, right={right}, mid={mid}, arr[mid]={arr[mid]}")
    if arr[mid] == target:
        print(f"Found at index {mid}!")
        break
    elif arr[mid] < target:
        left = mid + 1
    else:
        right = mid - 1
    step += 1

***Figure 11.4:** Each step halves the search space. For n=8 elements, at most 3 comparisons.*

**Listing 11.5 — Binary Search Recursive**

In [ ]:
def binary_search_recursive(arr, target, left=0, right=None):
    """
    Recursive binary search - O(log n) time, O(log n) space.
    Space is due to recursion stack.
    """
    if right is None:
        right = len(arr) - 1
    
    if left > right:
        return -1
    
    mid = (left + right) // 2
    
    if arr[mid] == target:
        return mid
    elif arr[mid] < target:
        return binary_search_recursive(arr, target, mid + 1, right)
    else:
        return binary_search_recursive(arr, target, left, mid - 1)

# Test
arr = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
print(f"Array: {arr}")
print(f"Index of 10: {binary_search_recursive(arr, 10)}")
print(f"Index of 15: {binary_search_recursive(arr, 15)}")
print(f"Index of 2: {binary_search_recursive(arr, 2)}")
print(f"Index of 20: {binary_search_recursive(arr, 20)}")

***Figure 11.5:** Recursive version is elegant but uses O(log n) stack space.*

---
## 4. Binary Search Variants


### Finding Boundaries

**Listing 11.6 — Lower and Upper Bound**

In [ ]:
def lower_bound(arr, target):
    """
    Find first index where arr[i] >= target.
    Returns len(arr) if all elements < target.
    """
    left, right = 0, len(arr)
    
    while left < right:
        mid = (left + right) // 2
        if arr[mid] < target:
            left = mid + 1
        else:
            right = mid
    
    return left

def upper_bound(arr, target):
    """
    Find first index where arr[i] > target.
    Returns len(arr) if all elements <= target.
    """
    left, right = 0, len(arr)
    
    while left < right:
        mid = (left + right) // 2
        if arr[mid] <= target:
            left = mid + 1
        else:
            right = mid
    
    return left

# Test with duplicates
arr = [1, 2, 2, 2, 3, 4, 5]
print(f"Array: {arr}")
print(f"\nFor target=2:")
print(f"  Lower bound (first >=): {lower_bound(arr, 2)}")
print(f"  Upper bound (first >):  {upper_bound(arr, 2)}")
print(f"  Count of 2s: {upper_bound(arr, 2) - lower_bound(arr, 2)}")

***Figure 11.6:** Lower/upper bounds find insertion points and handle duplicates.*

**Listing 11.7 — Find First and Last Position**

In [ ]:
def search_range(arr, target):
    """
    Find first and last position of target in sorted array.
    Returns [-1, -1] if not found.
    """
    def find_first(arr, target):
        left, right = 0, len(arr) - 1
        result = -1
        while left <= right:
            mid = (left + right) // 2
            if arr[mid] == target:
                result = mid
                right = mid - 1  # Keep searching left
            elif arr[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return result
    
    def find_last(arr, target):
        left, right = 0, len(arr) - 1
        result = -1
        while left <= right:
            mid = (left + right) // 2
            if arr[mid] == target:
                result = mid
                left = mid + 1  # Keep searching right
            elif arr[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return result
    
    return [find_first(arr, target), find_last(arr, target)]

# Test
arr = [5, 7, 7, 8, 8, 8, 10]
print(f"Array: {arr}")
print(f"Range of 8: {search_range(arr, 8)}")
print(f"Range of 7: {search_range(arr, 7)}")
print(f"Range of 6: {search_range(arr, 6)}")

***Figure 11.7:** Two binary searches find first and last occurrence of a value.*

### Finding Minimum/Maximum

**Listing 11.8 — Find Peak Element**

In [ ]:
def find_peak_element(arr):
    """
    Find a peak element where arr[i] > arr[i-1] and arr[i] > arr[i+1].
    Multiple peaks may exist; return any one.
    """
    left, right = 0, len(arr) - 1
    
    while left < right:
        mid = (left + right) // 2
        
        if arr[mid] > arr[mid + 1]:
            # Peak is on left side (or at mid)
            right = mid
        else:
            # Peak is on right side
            left = mid + 1
    
    return left

# Test
arr = [1, 2, 3, 1]
print(f"Array: {arr}")
print(f"Peak index: {find_peak_element(arr)}")
print(f"Peak value: {arr[find_peak_element(arr)]}")

arr = [1, 2, 1, 3, 5, 6, 4]
print(f"\nArray: {arr}")
idx = find_peak_element(arr)
print(f"Peak index: {idx}, value: {arr[idx]}")

***Figure 11.8:** Binary search finds peak by following the ascending slope.*

**Listing 11.9 — Find Minimum in Rotated Array**

In [ ]:
def find_min_rotated(arr):
    """
    Find minimum in rotated sorted array (no duplicates).
    Array was sorted, then rotated at some pivot.
    """
    left, right = 0, len(arr) - 1
    
    while left < right:
        mid = (left + right) // 2
        
        if arr[mid] > arr[right]:
            # Minimum is in right half
            left = mid + 1
        else:
            # Minimum is in left half (or at mid)
            right = mid
    
    return arr[left]

# Test
arr = [4, 5, 6, 7, 0, 1, 2]
print(f"Rotated array: {arr}")
print(f"Minimum: {find_min_rotated(arr)}")

arr = [3, 4, 5, 1, 2]
print(f"\nRotated array: {arr}")
print(f"Minimum: {find_min_rotated(arr)}")

***Figure 11.9:** In rotated array, compare mid with right to find which half contains minimum.*

---
## 5. Search in Rotated Arrays


**Listing 11.10 — Search in Rotated Array**

In [ ]:
def search_rotated(arr, target):
    """
    Search in rotated sorted array (no duplicates).
    One half is always sorted; use that to determine which half.
    """
    left, right = 0, len(arr) - 1
    
    while left <= right:
        mid = (left + right) // 2
        
        if arr[mid] == target:
            return mid
        
        # Left half is sorted
        if arr[left] <= arr[mid]:
            if arr[left] <= target < arr[mid]:
                right = mid - 1
            else:
                left = mid + 1
        # Right half is sorted
        else:
            if arr[mid] < target <= arr[right]:
                left = mid + 1
            else:
                right = mid - 1
    
    return -1

# Test
arr = [4, 5, 6, 7, 0, 1, 2]
print(f"Array: {arr}")
print(f"Index of 0: {search_rotated(arr, 0)}")
print(f"Index of 6: {search_rotated(arr, 6)}")
print(f"Index of 3: {search_rotated(arr, 3)}")

***Figure 11.10:** Determine which half is sorted, then check if target is in that half.*

**Listing 11.11 — Search in Rotated Array with Duplicates**

In [ ]:
def search_rotated_duplicates(arr, target):
    """
    Search in rotated sorted array WITH duplicates.
    Worst case O(n) when many duplicates.
    """
    left, right = 0, len(arr) - 1
    
    while left <= right:
        mid = (left + right) // 2
        
        if arr[mid] == target:
            return True
        
        # Handle duplicates - shrink search space
        if arr[left] == arr[mid] == arr[right]:
            left += 1
            right -= 1
        # Left half is sorted
        elif arr[left] <= arr[mid]:
            if arr[left] <= target < arr[mid]:
                right = mid - 1
            else:
                left = mid + 1
        # Right half is sorted
        else:
            if arr[mid] < target <= arr[right]:
                left = mid + 1
            else:
                right = mid - 1
    
    return False

# Test
arr = [2, 5, 6, 0, 0, 1, 2]
print(f"Array: {arr}")
print(f"Contains 0: {search_rotated_duplicates(arr, 0)}")
print(f"Contains 3: {search_rotated_duplicates(arr, 3)}")

arr = [1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1]
print(f"\nArray with many duplicates: {arr}")
print(f"Contains 2: {search_rotated_duplicates(arr, 2)}")

***Figure 11.11:** Duplicates require handling the case where left == mid == right.*

---
## 6. Search in 2D Arrays


**Listing 11.12 — Search 2D Matrix (Sorted Rows)**

In [ ]:
def search_matrix(matrix, target):
    """
    Search in 2D matrix where:
    - Each row is sorted
    - First element of each row > last element of previous row
    Treat as flattened sorted array.
    """
    if not matrix or not matrix[0]:
        return False
    
    rows, cols = len(matrix), len(matrix[0])
    left, right = 0, rows * cols - 1
    
    while left <= right:
        mid = (left + right) // 2
        # Convert 1D index to 2D
        row, col = mid // cols, mid % cols
        val = matrix[row][col]
        
        if val == target:
            return True
        elif val < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return False

# Test
matrix = [
    [1,  3,  5,  7],
    [10, 11, 16, 20],
    [23, 30, 34, 60]
]
print("Matrix:")
for row in matrix:
    print(f"  {row}")

print(f"\nSearch 3: {search_matrix(matrix, 3)}")
print(f"Search 13: {search_matrix(matrix, 13)}")

***Figure 11.12:** Fully sorted 2D matrix can be treated as 1D array for binary search.*

**Listing 11.13 — Search 2D Matrix II (Row/Col Sorted)**

In [ ]:
def search_matrix_ii(matrix, target):
    """
    Search in 2D matrix where:
    - Each row is sorted left to right
    - Each column is sorted top to bottom
    Start from top-right (or bottom-left) corner.
    """
    if not matrix or not matrix[0]:
        return False
    
    rows, cols = len(matrix), len(matrix[0])
    row, col = 0, cols - 1  # Start top-right
    
    while row < rows and col >= 0:
        if matrix[row][col] == target:
            return True
        elif matrix[row][col] > target:
            col -= 1  # Go left
        else:
            row += 1  # Go down
    
    return False

# Test
matrix = [
    [1,  4,  7,  11, 15],
    [2,  5,  8,  12, 19],
    [3,  6,  9,  16, 22],
    [10, 13, 14, 17, 24],
    [18, 21, 23, 26, 30]
]
print("Matrix (rows and columns sorted):")
for row in matrix:
    print(f"  {row}")

print(f"\nSearch 5: {search_matrix_ii(matrix, 5)}")
print(f"Search 20: {search_matrix_ii(matrix, 20)}")

***Figure 11.13:** Start from corner—each step eliminates a row or column. O(m + n) time.*

**Listing 11.14 — Kth Smallest in Sorted Matrix**

In [ ]:
def kth_smallest_matrix(matrix, k):
    """
    Find kth smallest element in row/column sorted matrix.
    Binary search on value range, count elements <= mid.
    """
    n = len(matrix)
    
    def count_less_equal(target):
        """Count elements <= target using staircase search."""
        count = 0
        row, col = n - 1, 0  # Start bottom-left
        while row >= 0 and col < n:
            if matrix[row][col] <= target:
                count += row + 1  # All elements in this column
                col += 1
            else:
                row -= 1
        return count
    
    # Binary search on value
    left, right = matrix[0][0], matrix[n-1][n-1]
    
    while left < right:
        mid = (left + right) // 2
        if count_less_equal(mid) < k:
            left = mid + 1
        else:
            right = mid
    
    return left

# Test
matrix = [
    [1,  5,  9],
    [10, 11, 13],
    [12, 13, 15]
]
print("Matrix:")
for row in matrix:
    print(f"  {row}")

for k in [1, 5, 8]:
    print(f"  {k}th smallest: {kth_smallest_matrix(matrix, k)}")

***Figure 11.14:** Binary search on value range combined with counting achieves O(n log(max-min)).*

---
## 7. Search in Trees


**Listing 11.15 — BST Search**

In [ ]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def search_bst(root, target):
    """
    Search in Binary Search Tree - O(h) time where h is height.
    O(log n) for balanced tree, O(n) for skewed tree.
    """
    while root:
        if target == root.val:
            return root
        elif target < root.val:
            root = root.left
        else:
            root = root.right
    return None

def search_bst_recursive(root, target):
    """Recursive BST search."""
    if not root or root.val == target:
        return root
    if target < root.val:
        return search_bst_recursive(root.left, target)
    return search_bst_recursive(root.right, target)

# Build BST:     5
#              /   \
#             3     7
#            / \   / \
#           2   4 6   8

root = TreeNode(5)
root.left = TreeNode(3, TreeNode(2), TreeNode(4))
root.right = TreeNode(7, TreeNode(6), TreeNode(8))

print("BST: [5, 3, 7, 2, 4, 6, 8]")
for target in [4, 9]:
    result = search_bst(root, target)
    print(f"Search {target}: {'Found' if result else 'Not found'}")

***Figure 11.15:** BST search follows left/right based on comparison—like binary search in arrays.*

**Listing 11.16 — Find Closest Value in BST**

In [ ]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def closest_value(root, target):
    """Find value in BST closest to target."""
    closest = root.val
    
    while root:
        if abs(root.val - target) < abs(closest - target):
            closest = root.val
        
        if target < root.val:
            root = root.left
        elif target > root.val:
            root = root.right
        else:
            return root.val  # Exact match
    
    return closest

# Build BST
root = TreeNode(4)
root.left = TreeNode(2, TreeNode(1), TreeNode(3))
root.right = TreeNode(5)

print("BST: [4, 2, 5, 1, 3]")
for target in [3.7, 2.5, 4.5, 0.1]:
    print(f"Closest to {target}: {closest_value(root, target)}")

***Figure 11.16:** Track closest seen while traversing towards target.*

**Listing 11.17 — Validate BST**

In [ ]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def is_valid_bst(root, min_val=float('-inf'), max_val=float('inf')):
    """
    Validate Binary Search Tree.
    Each node must be within valid range based on ancestors.
    """
    if not root:
        return True
    
    if root.val <= min_val or root.val >= max_val:
        return False
    
    return (is_valid_bst(root.left, min_val, root.val) and
            is_valid_bst(root.right, root.val, max_val))

# Valid BST
root1 = TreeNode(2, TreeNode(1), TreeNode(3))
print(f"[2, 1, 3] is valid BST: {is_valid_bst(root1)}")

# Invalid BST
root2 = TreeNode(5, TreeNode(1), TreeNode(4, TreeNode(3), TreeNode(6)))
print(f"[5, 1, 4, null, null, 3, 6] is valid BST: {is_valid_bst(root2)}")

***Figure 11.17:** Pass valid range to each subtree—left must be less than root, right must be greater.*

---
## 8. Search in Graphs


**Listing 11.18 — BFS Shortest Path**

In [ ]:
from collections import deque

def bfs_shortest_path(graph, start, target):
    """
    BFS finds shortest path in unweighted graph.
    Returns path and distance, or (None, -1) if not found.
    """
    if start == target:
        return [start], 0
    
    visited = {start}
    queue = deque([(start, [start])])
    
    while queue:
        node, path = queue.popleft()
        
        for neighbor in graph[node]:
            if neighbor == target:
                return path + [neighbor], len(path)
            
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))
    
    return None, -1

# Test
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

path, dist = bfs_shortest_path(graph, 'A', 'F')
print(f"Shortest A to F: {path}, distance: {dist}")

path, dist = bfs_shortest_path(graph, 'D', 'F')
print(f"Shortest D to F: {path}, distance: {dist}")

***Figure 11.18:** BFS explores level by level, guaranteeing shortest path in unweighted graphs.*

**Listing 11.19 — DFS Path Existence**

In [ ]:
def dfs_path_exists(graph, start, target, visited=None):
    """
    DFS checks if path exists between two nodes.
    Returns True/False (not shortest path).
    """
    if visited is None:
        visited = set()
    
    if start == target:
        return True
    
    visited.add(start)
    
    for neighbor in graph.get(start, []):
        if neighbor not in visited:
            if dfs_path_exists(graph, neighbor, target, visited):
                return True
    
    return False

# Test
graph = {
    'A': ['B', 'C'],
    'B': ['D'],
    'C': ['E'],
    'D': [],
    'E': ['F'],
    'F': []
}

print(f"Path A to F exists: {dfs_path_exists(graph, 'A', 'F')}")
print(f"Path A to D exists: {dfs_path_exists(graph, 'A', 'D')}")
print(f"Path D to A exists: {dfs_path_exists(graph, 'D', 'A')}")

***Figure 11.19:** DFS uses recursion/stack to explore paths deeply before backtracking.*

**Listing 11.20 — Binary Search on Answer**

In [ ]:
def can_finish_in_days(weights, capacity, days):
    """Check if we can ship all weights within given days."""
    current_load = 0
    days_needed = 1
    
    for w in weights:
        if current_load + w > capacity:
            days_needed += 1
            current_load = w
        else:
            current_load += w
    
    return days_needed <= days

def ship_within_days(weights, days):
    """
    Find minimum ship capacity to ship all packages within days.
    Binary search on the answer (capacity).
    """
    # Minimum: largest single package
    # Maximum: sum of all packages
    left = max(weights)
    right = sum(weights)
    
    while left < right:
        mid = (left + right) // 2
        if can_finish_in_days(weights, mid, days):
            right = mid
        else:
            left = mid + 1
    
    return left

# Test
weights = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
days = 5
print(f"Weights: {weights}")
print(f"Days: {days}")
print(f"Minimum capacity: {ship_within_days(weights, days)}")

***Figure 11.20:** Binary search on answer: search the solution space, not the data.*

---
## 9. Python's bisect Module


**Listing 11.21 — bisect_left and bisect_right**

In [ ]:
import bisect

arr = [1, 2, 2, 2, 3, 4, 5]
print(f"Array: {arr}")

# bisect_left: insertion point for leftmost position
print(f"\nbisect_left(arr, 2): {bisect.bisect_left(arr, 2)}")
print(f"bisect_left(arr, 2.5): {bisect.bisect_left(arr, 2.5)}")

# bisect_right (or bisect): insertion point for rightmost position
print(f"\nbisect_right(arr, 2): {bisect.bisect_right(arr, 2)}")
print(f"bisect(arr, 2): {bisect.bisect(arr, 2)}")  # Same as bisect_right

# Count occurrences using both
def count_occurrences(arr, x):
    return bisect.bisect_right(arr, x) - bisect.bisect_left(arr, x)

print(f"\nCount of 2: {count_occurrences(arr, 2)}")

# Find exact match (or -1)
def binary_search_bisect(arr, x):
    i = bisect.bisect_left(arr, x)
    if i < len(arr) and arr[i] == x:
        return i
    return -1

print(f"Index of 3: {binary_search_bisect(arr, 3)}")
print(f"Index of 6: {binary_search_bisect(arr, 6)}")

***Figure 11.21:** bisect_left finds first position, bisect_right finds after last position.*

**Listing 11.22 — insort for Sorted Insertion**

In [ ]:
import bisect

# insort maintains sorted order during insertion
arr = [1, 3, 5, 7, 9]
print(f"Original: {arr}")

bisect.insort(arr, 4)
print(f"After insort(4): {arr}")

bisect.insort(arr, 0)
print(f"After insort(0): {arr}")

bisect.insort(arr, 10)
print(f"After insort(10): {arr}")

# insort_left vs insort_right for duplicates
arr = [1, 2, 2, 2, 3]
bisect.insort_left(arr, 2)  # Insert before existing 2s
print(f"\ninsort_left(2): {arr}")

arr = [1, 2, 2, 2, 3]
bisect.insort_right(arr, 2)  # Insert after existing 2s
print(f"insort_right(2): {arr}")

***Figure 11.22:** insort combines finding position and inserting in one call.*

**Listing 11.23 — Custom Key with bisect**

In [ ]:
import bisect

# Sort by custom key - need to maintain separate key list
data = [(1, 'a'), (3, 'c'), (5, 'e'), (7, 'g')]
keys = [x[0] for x in data]  # Extract keys

# Find where to insert (4, 'd')
new_item = (4, 'd')
idx = bisect.bisect_left(keys, new_item[0])
data.insert(idx, new_item)
keys.insert(idx, new_item[0])

print(f"After inserting {new_item}: {data}")

# Python 3.10+ has key parameter
# bisect.bisect_left(data, 4, key=lambda x: x[0])

# Alternative: use a wrapper class
class KeyWrapper:
    def __init__(self, item, key):
        self.item = item
        self.key = key(item)
    def __lt__(self, other):
        return self.key < other.key

# Example with objects sorted by attribute
students = [('Alice', 85), ('Bob', 90), ('Carol', 78)]
students.sort(key=lambda x: x[1])
print(f"\nStudents by grade: {students}")

***Figure 11.23:** Custom keys require maintaining a separate key list or using a wrapper.*

---
## 10. Common Patterns


### Pattern 1: Binary Search Template

**Listing 11.24 — Universal Binary Search Template**

In [ ]:
def binary_search_template(arr, target):
    """
    Universal template for binary search problems.
    Find minimum index where condition(mid) is True.
    """
    left, right = 0, len(arr)  # [left, right) search space
    
    def condition(mid):
        # Define based on problem
        return arr[mid] >= target
    
    while left < right:
        mid = left + (right - left) // 2
        if condition(mid):
            right = mid
        else:
            left = mid + 1
    
    return left

# Example: Find first element >= target (lower bound)
arr = [1, 3, 5, 7, 9, 11]
print(f"Array: {arr}")
print(f"First >= 6: index {binary_search_template(arr, 6)}")
print(f"First >= 7: index {binary_search_template(arr, 7)}")

# The template can be adapted:
# - For exact match: check arr[result] == target
# - For upper bound: change condition to arr[mid] > target
# - For last occurrence: change to find first > target, then subtract 1

***Figure 11.24:** The template finds the boundary where condition changes from False to True.*

### Pattern 2: Search on Answer

**Listing 11.25 — Split Array Largest Sum**

In [ ]:
def split_array(nums, k):
    """
    Split array into k subarrays to minimize largest sum.
    Binary search on the answer (max sum).
    """
    def can_split(max_sum):
        """Can we split into <= k subarrays with this max_sum?"""
        count = 1
        current_sum = 0
        for num in nums:
            if current_sum + num > max_sum:
                count += 1
                current_sum = num
            else:
                current_sum += num
        return count <= k
    
    left = max(nums)  # At least largest element
    right = sum(nums)  # At most entire array
    
    while left < right:
        mid = (left + right) // 2
        if can_split(mid):
            right = mid
        else:
            left = mid + 1
    
    return left

# Test
nums = [7, 2, 5, 10, 8]
k = 2
print(f"Array: {nums}, k={k}")
print(f"Minimum largest sum: {split_array(nums, k)}")
# Split: [7, 2, 5] and [10, 8] with sums 14 and 18

***Figure 11.25:** When asked to minimize/maximize something, binary search on that value.*

### Pattern 3: Two Pointers + Binary Search

**Listing 11.26 — Find K Closest Elements**

In [ ]:
def find_closest_elements(arr, k, x):
    """
    Find k closest elements to x in sorted array.
    Binary search + two pointers.
    """
    # Binary search for window start
    left, right = 0, len(arr) - k
    
    while left < right:
        mid = (left + right) // 2
        # Compare distances at window boundaries
        if x - arr[mid] > arr[mid + k] - x:
            left = mid + 1
        else:
            right = mid
    
    return arr[left:left + k]

# Test
arr = [1, 2, 3, 4, 5]
print(f"Array: {arr}")
print(f"3 closest to 3: {find_closest_elements(arr, 3, 3)}")
print(f"3 closest to -1: {find_closest_elements(arr, 3, -1)}")

arr = [1, 1, 1, 10, 10, 10]
print(f"\nArray: {arr}")
print(f"3 closest to 9: {find_closest_elements(arr, 3, 9)}")

***Figure 11.26:** Binary search finds optimal window position by comparing distances.*

### Pattern 4: Exponential Search

**Listing 11.27 — Exponential Search**

In [ ]:
def exponential_search(arr, target):
    """
    Find target using exponential search.
    Good when target is near beginning of large array.
    """
    if arr[0] == target:
        return 0
    
    # Find range where target might exist
    bound = 1
    while bound < len(arr) and arr[bound] < target:
        bound *= 2
    
    # Binary search in found range
    left = bound // 2
    right = min(bound, len(arr) - 1)
    
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1

# Test
arr = list(range(1000))
print(f"Array: [0, 1, 2, ..., 999]")
print(f"Index of 5: {exponential_search(arr, 5)}")
print(f"Index of 500: {exponential_search(arr, 500)}")

***Figure 11.27:** Exponential search doubles bound until exceeding target, then binary searches.*

### Pattern 5: Ternary Search

**Listing 11.28 — Ternary Search for Unimodal Function**

In [ ]:
def ternary_search_max(f, left, right, epsilon=1e-9):
    """
    Find maximum of unimodal function.
    Function first increases then decreases.
    """
    while right - left > epsilon:
        m1 = left + (right - left) / 3
        m2 = right - (right - left) / 3
        
        if f(m1) < f(m2):
            left = m1
        else:
            right = m2
    
    return (left + right) / 2

# Test with parabola y = -(x-3)^2 + 10 (max at x=3)
f = lambda x: -(x - 3) ** 2 + 10

max_x = ternary_search_max(f, 0, 10)
print(f"Function: -(x-3)^2 + 10")
print(f"Maximum at x = {max_x:.6f}")
print(f"Maximum value = {f(max_x):.6f}")

***Figure 11.28:** Ternary search finds extrema in unimodal functions by eliminating 1/3 each step.*

- **Linear search:** O(n), works on any data, simple but slow
- **Binary search:** O(log n), requires sorted array with random access
- **Search on answer:** Binary search the solution space, not just data
- **BST search:** O(h) where h is height, O(log n) for balanced trees
- **Graph search:** BFS for shortest path, DFS for path existence
- **bisect module:** Production-ready binary search in Python

---
## 11. Common Pitfalls


### Pitfall 1: Off-by-One Errors

**Listing 11.29 — Binary Search Boundary Conditions**

In [ ]:
# Three common binary search patterns:

def pattern1_exact(arr, target):
    """Find exact match: left <= right, mid±1"""
    left, right = 0, len(arr) - 1
    while left <= right:  # <= for inclusive
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1

def pattern2_lower(arr, target):
    """Find lower bound: left < right, right = mid"""
    left, right = 0, len(arr)  # right = len for insertion point
    while left < right:  # < because right is exclusive
        mid = (left + right) // 2
        if arr[mid] < target:
            left = mid + 1
        else:
            right = mid  # Don't skip mid
    return left

def pattern3_last(arr, target):
    """Find last occurrence: modify lower bound"""
    left, right = 0, len(arr)
    while left < right:
        mid = (left + right) // 2
        if arr[mid] <= target:
            left = mid + 1
        else:
            right = mid
    return left - 1 if left > 0 and arr[left-1] == target else -1

arr = [1, 2, 2, 2, 3, 4]
print(f"Array: {arr}")
print(f"Exact match for 2: {pattern1_exact(arr, 2)}")
print(f"Lower bound for 2: {pattern2_lower(arr, 2)}")
print(f"Last occurrence of 2: {pattern3_last(arr, 2)}")

***Figure 11.29:** Different patterns require different boundary conditions—be consistent!*

### Pitfall 2: Integer Overflow (in other languages)

**Listing 11.30 — Safe Mid Calculation**

In [ ]:
# In Python, integers have arbitrary precision - no overflow
# But in C/Java, (left + right) can overflow!

# WRONG in C/Java (but fine in Python):
# mid = (left + right) / 2

# SAFE in all languages:
# mid = left + (right - left) // 2

# Or using bit manipulation:
# mid = (left + right) >> 1  # Still can overflow in C/Java
# mid = left + ((right - left) >> 1)  # Safe

# Python example showing both work:
left = 10**18
right = 10**18 + 100

mid1 = (left + right) // 2
mid2 = left + (right - left) // 2

print(f"left = {left}")
print(f"right = {right}")
print(f"(left + right) // 2 = {mid1}")
print(f"left + (right - left) // 2 = {mid2}")
print(f"Both equal: {mid1 == mid2}")

***Figure 11.30:** Use left + (right - left) // 2 for portability to other languages.*

### Pitfall 3: Infinite Loop

**Listing 11.31 — Avoiding Infinite Loops**

In [ ]:
# WRONG: Can cause infinite loop
def bad_binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left < right:
        mid = (left + right) // 2
        if arr[mid] < target:
            left = mid  # WRONG! Should be mid + 1
        else:
            right = mid
    return left

# When left = 0, right = 1, mid = 0
# If arr[0] < target, left stays 0 forever!

# RIGHT: Ensure progress each iteration
def good_binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left < right:
        mid = (left + right) // 2
        if arr[mid] < target:
            left = mid + 1  # CORRECT: Always makes progress
        else:
            right = mid
    return left

print("Testing on [1, 3, 5, 7]:")
arr = [1, 3, 5, 7]
print(f"Search for 4: index {good_binary_search(arr, 4)}")

# Rule: At least one of left or right must change each iteration

***Figure 11.31:** When left < right, use left = mid + 1 to ensure progress.*

### Pitfall 4: Searching Unsorted Array

**Listing 11.32 — Verify Sorted Before Binary Search**

In [ ]:
def binary_search_safe(arr, target):
    """Binary search with sorted check (for debugging)."""
    # Verify sorted (O(n) - only for debugging!)
    assert all(arr[i] <= arr[i+1] for i in range(len(arr)-1)), \
           "Array must be sorted!"
    
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1

# Test with sorted array
arr = [1, 3, 5, 7, 9]
print(f"Sorted array: {arr}")
print(f"Search 5: {binary_search_safe(arr, 5)}")

# Test with unsorted array
arr = [3, 1, 4, 1, 5]
print(f"\nUnsorted array: {arr}")
try:
    binary_search_safe(arr, 4)
except AssertionError as e:
    print(f"Error: {e}")

***Figure 11.32:** Binary search on unsorted array gives incorrect results—verify during debugging.*

### Pitfall 5: Empty Array

**Listing 11.33 — Handle Edge Cases**

In [ ]:
def binary_search_robust(arr, target):
    """Binary search handling edge cases."""
    # Handle empty array
    if not arr:
        return -1
    
    # Handle single element
    if len(arr) == 1:
        return 0 if arr[0] == target else -1
    
    left, right = 0, len(arr) - 1
    
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1

# Test edge cases
print(f"Empty array []: {binary_search_robust([], 5)}")
print(f"Single [5] search 5: {binary_search_robust([5], 5)}")
print(f"Single [5] search 3: {binary_search_robust([5], 3)}")
print(f"Two elements [1,3] search 3: {binary_search_robust([1,3], 3)}")

***Figure 11.33:** Always handle empty arrays and single-element arrays explicitly.*

---
# 📝 Exercises


### Exercise 1: Search Insert Position  (⭐ Easy)

Given a sorted array and target, return index if found, else the index where it would be inserted to keep sorted order.

**Expected:** (Expected: [1,3,5,6], target=5 → 2)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Lower Bound:**

                        This is finding the first position where `arr[i] >= target`.
- **Hint 2 - Search Range:**

                        Use `left = 0`, `right = len(nums)` (note: right can be past end).
- **Hint 3 - Movement:**

                        If `nums[mid] < target`, move `left = mid + 1`. Otherwise `right = mid`.
</details>

In [ ]:
# ✏️ [EX1]
# Search Insert Position - Lower bound

def search_insert(nums, target):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(search_insert([1,3,5,6], 5))  # 2
# print(search_insert([1,3,5,6], 2))  # 1
# print(search_insert([1,3,5,6], 7))  # 4

### Exercise 2: Find First Bad Version  (⭐ Easy)

You have n versions [1, 2, ..., n]. Given a function isBadVersion(v) that returns True if v is bad, find the first bad version. All versions after a bad version are also bad.

**Expected:** (Expected: Find boundary where False→True)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Search Space:**

                        Search space is [1, n]. Versions are 1-indexed.
- **Hint 2 - Mid Check:**

                        If `isBadVersion(mid)` is True, first bad is at or before mid.
- **Hint 3 - Movement:**

                        If bad → `right = mid`. If good → `left = mid + 1`.
</details>

In [ ]:
# ✏️ [EX2]
# Find First Bad Version - Boundary search

def first_bad_version(n, is_bad_version):
    # Your code here
    pass

# Test your implementation (uncomment)
# def is_bad(v): return v >= 4  # 4 is first bad
# print(first_bad_version(5, is_bad))  # 4

### Exercise 3: Sqrt(x)  (⭐⭐ Medium)

Compute the square root of x rounded down to the nearest integer. Do not use built-in sqrt functions.

**Expected:** (Expected: sqrt(8) → 2)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Search Space:**

                        Search space is [0, x]. Find largest n where n*n <= x.
- **Hint 2 - Valid Answer:**

                        If `mid*mid <= x`, mid is a valid answer. Track best seen.
- **Hint 3 - Movement:**

                        If valid, try larger: `left = mid + 1`. If too big: `right = mid - 1`.
</details>

In [ ]:
# ✏️ [EX3]
# Sqrt(x) - Binary search for square root

def my_sqrt(x):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(my_sqrt(4))   # 2
# print(my_sqrt(8))   # 2 (sqrt(8) ≈ 2.83)
# print(my_sqrt(16))  # 4

### Exercise 4: Find Minimum in Rotated Sorted Array II  (⭐⭐⭐ Hard)

Find minimum in rotated sorted array that may contain duplicates.

**Expected:** (Expected: [2,2,2,0,1] → 0)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Compare with Right:**

                        Compare `nums[mid]` with `nums[right]` to determine which half.
- **Hint 2 - Duplicates Problem:**

                        When `nums[mid] == nums[right]`, can't determine side. Just do `right -= 1`.
- **Hint 3 - Movement:**

                        If `mid > right`: min on right. If `mid < right`: min on left or at mid.
</details>

In [ ]:
# ✏️ [EX4]
# Find Minimum in Rotated Array with Duplicates

def find_min_duplicates(nums):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(find_min_duplicates([2,2,2,0,1]))  # 0
# print(find_min_duplicates([1,1,1,1,1]))  # 1

### Exercise 5: Koko Eating Bananas  (⭐⭐⭐ Hard)

Koko has n piles of bananas. She can eat k bananas per hour. Each hour she chooses a pile and eats k bananas (or the whole pile if less than k). Find minimum k to finish within h hours.

**Expected:** (Expected: piles=[3,6,7,11], h=8 → k=4)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Search Space:**

                        k ranges from 1 to max(piles). Binary search for minimum valid k.
- **Hint 2 - Hours Calculation:**

                        Hours for pile p at speed k: `(p + k - 1) // k` (ceiling division).
- **Hint 3 - Check Function:**

                        Sum hours for all piles. If total <= h, k is valid (try smaller). Otherwise try larger.
</details>

In [ ]:
# ✏️ [EX5]
# Koko Eating Bananas - Binary search on answer

def min_eating_speed(piles, h):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(min_eating_speed([3,6,7,11], 8))  # 4
# print(min_eating_speed([30,11,23,4,20], 5))  # 30

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,10}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-10 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request

WEEK = "Week_11"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ── Check info was filled in ──
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Run the cell above first to set your info!")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Run the cell above first — some fields are empty.")

# ── Extract exercise answers from IPython history ──
_answers = {}
try:
    _ipy = get_ipython()
    _hist = _ipy.history_manager.get_range(output=False)
    for _sess, _line, _src in _hist:
        _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
        if _m:
            _ex_id = "ex" + _m.group(1)
            _lines = _src.split("\n")
            _clean = "\n".join(_lines[1:]).strip()
            _answers[_ex_id] = {
                "code": _clean,
                "modified": len(_clean) > 5
            }
except Exception:
    pass

# ── Fallback: also check In[] from current session ──
if not _answers:
    try:
        for _src in In:
            if not _src:
                continue
            _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
            if _m:
                _ex_id = "ex" + _m.group(1)
                _lines = _src.split("\n")
                _clean = "\n".join(_lines[1:]).strip()
                _answers[_ex_id] = {
                    "code": _clean,
                    "modified": len(_clean) > 5
                }
    except NameError:
        pass

# ── Fallback: try reading notebook file (VS Code) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir(".") if f.endswith(".ipynb") and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), "r", encoding="utf-8") as _f:
            _nb = json.load(_f)
        for _cell in _nb["cells"]:
            if _cell["cell_type"] != "code":
                continue
            _src = "".join(_cell["source"]) if isinstance(_cell["source"], list) else _cell["source"]
            _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
            if _m:
                _ex_id = "ex" + _m.group(1)
                _lines = _src.split("\n")
                _clean = "\n".join(_lines[1:]).strip()
                _answers[_ex_id] = {
                    "code": _clean,
                    "modified": len(_clean) > 5
                }

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys()))}")

if not _answers:
    print("\n⚠️  No exercise answers found!")
    print("Make sure you RAN all exercise cells before submitting.")
    raise SystemExit()

# ── Send ──
_data = json.dumps({
    "week": WEEK,
    "studentId": _sid,
    "studentName": _sname,
    "studentEmail": _semail,
    "classCode": _scode,
    "source": "dsa-notebook",
    "timeOnPage": 0,
    "answers": _answers
}).encode("utf-8")

print("📡 Submitting...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get("success"):
        print(f"\n✅ {_result['message']}")
        print("📧 Check your email for confirmation.")
    else:
        print(f"\n❌ {_result.get('message', 'Submission failed')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  Request sent — check your email for confirmation.")
    print(f"(If no email arrives, try again or contact your instructor)")
